In [122]:
import spotipy
#import spotipy.util as util
from zuzana_config import Client_ID, Client_Secret
import pandas as pd

In [123]:
token = spotipy.oauth2.SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_Secret)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

C:\Users\zuzan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


In [124]:
# get information on playlist: Today's Top Hits by Spotify
username = "spotify"
playlist_id = "37i9dQZF1DXcBWIGoYBM5M"

#sp.user_playlist_tracks(username, playlist_id)

In [127]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name","track_id","danceability","energy","key","loudness","mode","speechiness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["popularity"] = track["track"]["popularity"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [128]:
analyze_playlist(username, playlist_id)

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity
0,DaBaby,BLAME IT ON BABY,ROCKSTAR (feat. Roddy Ricch),7ytR5pFWmSjzHJIeQkgog4,0.746,0.690,11,-7.956,1,0.1640,0,0.1010,0.4970,89.977,181733,4,100.0
1,DJ Khaled,POPSTAR (feat. Drake),POPSTAR (feat. Drake),6EDO9iiTtwNv6waLwa1UUq,0.800,0.560,5,-4.818,0,0.2610,0,0.1340,0.4500,163.071,200221,4,86.0
2,Harry Styles,Fine Line,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,0.548,0.816,0,-4.209,1,0.0465,0,0.3350,0.5570,95.390,174000,4,95.0
3,Jawsh 685,Savage Love (Laxed - Siren Beat),Savage Love (Laxed - Siren Beat),5T490vvoFNU6psep0NPmxs,0.795,0.480,3,-8.556,1,0.0688,0,0.2710,0.7230,149.919,171375,4,87.0
4,Jack Harlow,"WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...","WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...",2MbdDtCv5LUVjYy9RuGTgC,0.904,0.723,11,-5.224,0,0.2600,0,0.1850,0.8350,145.013,227478,4,90.0
5,Juice WRLD,Legends Never Die,Wishing Well,6o3QUC5oAE4g6WxRIFcZtb,0.650,0.613,9,-6.130,0,0.1280,0,0.2670,0.0804,149.972,194621,4,91.0
6,Topic,Breaking Me,Breaking Me,3H7ihDc1dqLriiWXwsc2po,0.789,0.720,8,-5.652,0,0.2180,0,0.1290,0.6640,122.031,166794,4,95.0
7,Dua Lipa,Break My Heart,Break My Heart,1raaNykBg1bDnWENUiglUA,0.730,0.729,4,-3.434,0,0.0886,1.39e-06,0.3490,0.4670,113.012,221820,4,88.0
8,StaySolidRocky,Party Girl,Party Girl,5RqR4ZCCKJDcBLIn4sih9l,0.728,0.431,6,-9.966,0,0.0622,0,0.0996,0.6290,130.022,147800,4,94.0
9,Juice WRLD,Legends Never Die,Come & Go (with Marshmello),7y7w4tl4MaRC2UMEj1mPtr,0.625,0.814,0,-5.181,1,0.0657,0,0.1580,0.5350,144.991,205485,4,91.0


In [129]:
playlist_df.head()

,artist,album,track_name,track_id,danceability,energy,loudness,instrumentalness,liveness,valence,tempo
0,DaBaby,BLAME IT ON BABY,ROCKSTAR (feat. Roddy Ricch),7ytR5pFWmSjzHJIeQkgog4,0.746,0.690,-7.956,0,0.101,0.497,89.977
1,DJ Khaled,POPSTAR (feat. Drake),POPSTAR (feat. Drake),6EDO9iiTtwNv6waLwa1UUq,0.800,0.560,-4.818,0,0.134,0.450,163.071
2,Harry Styles,Fine Line,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,0.548,0.816,-4.209,0,0.335,0.557,95.390
3,Jawsh 685,Savage Love (Laxed - Siren Beat),Savage Love (Laxed - Siren Beat),5T490vvoFNU6psep0NPmxs,0.795,0.480,-8.556,0,0.271,0.723,149.919
4,Jack Harlow,"WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...","WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...",2MbdDtCv5LUVjYy9RuGTgC,0.904,0.723,-5.224,0,0.185,0.835,145.013


In [130]:
playlist_df.to_csv("today_top_hits_spotify2.csv", index = False)

In [131]:
def analyze_playlist_dict(playlist_dict):
    
    # Loop through every playlist in the dict and analyze it
    for i, (key, val) in enumerate(playlist_dict.items()):
        playlist_df = analyze_playlist(*val)
        # Add a playlist column so that we can see which playlist a track belongs too
        playlist_df["playlist"] = key
        # Create or concat df
        if i == 0:
            playlist_dict_df = playlist_df
        else:
            playlist_dict_df = pd.concat([playlist_dict_df, playlist_df], ignore_index = True)
            
    return playlist_dict_df

In [132]:
usernames = {"spotify", "indiemono", "Susan Doles"}
for username in usernames:
    print(username)

Susan Doles
indiemono
spotify


In [133]:
playlist_ids ={"37i9dQZF1DX5IDTimEWoTd", "5KbTzqKBqxQRD8OBtJTZrS", "7sAUK3XK8NHH1s5vGcTBkF"}
playlist_ids

{'37i9dQZF1DX5IDTimEWoTd', '5KbTzqKBqxQRD8OBtJTZrS', '7sAUK3XK8NHH1s5vGcTBkF'}

In [187]:
playlist_dict = {"today_top_hits" : ("spotify", "37i9dQZF1DXcBWIGoYBM5M"), 
                 "billboard_100" : ("Billboard", "6UeSakyzhiEt4NB3UAd6NQ"),            
                 }
#"top_100_most_streamed" : ("jonathan.holgersson", "5ABHKGoOzxkaa28ttQV9sE")

In [188]:
playlist_dict

{'today_top_hits': ('spotify', '37i9dQZF1DXcBWIGoYBM5M'),
 'billboard_100': ('Billboard', '6UeSakyzhiEt4NB3UAd6NQ')}

In [189]:
#playlist_names = []
#for i in playlist_ids:
#    playlist_names.append(sp.playlist(i)['name'].replace(" ","_"))
#playlist_names

In [190]:
multiple_playlist_df = analyze_playlist_dict(playlist_dict)
multiple_playlist_df["playlist"].value_counts()

billboard_100     100
today_top_hits     50
Name: playlist, dtype: int64

In [192]:
multiple_playlist_df

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,playlist
0,DaBaby,BLAME IT ON BABY,ROCKSTAR (feat. Roddy Ricch),7ytR5pFWmSjzHJIeQkgog4,0.746,0.690,11,-7.956,1,0.1640,0,0.1010,0.497,89.977,181733,4,100.0,today_top_hits
1,DJ Khaled,POPSTAR (feat. Drake),POPSTAR (feat. Drake),6EDO9iiTtwNv6waLwa1UUq,0.800,0.560,5,-4.818,0,0.2610,0,0.1340,0.450,163.071,200221,4,86.0,today_top_hits
2,Harry Styles,Fine Line,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,0.548,0.816,0,-4.209,1,0.0465,0,0.3350,0.557,95.390,174000,4,95.0,today_top_hits
3,Jawsh 685,Savage Love (Laxed - Siren Beat),Savage Love (Laxed - Siren Beat),5T490vvoFNU6psep0NPmxs,0.795,0.480,3,-8.556,1,0.0688,0,0.2710,0.723,149.919,171375,4,87.0,today_top_hits
4,Jack Harlow,"WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...","WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...",2MbdDtCv5LUVjYy9RuGTgC,0.904,0.723,11,-5.224,0,0.2600,0,0.1850,0.835,145.013,227478,4,90.0,today_top_hits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Kenny Chesney,Knowing You,Here And Now,02LPxsehaIst1MW8wLtiyt,0.509,0.814,9,-3.576,1,0.0532,0,0.1200,0.712,148.034,171493,4,23.0,billboard_100
146,Lil Durk,3 Headed Goat (feat. Lil Baby & Polo G),3 Headed Goat (feat. Lil Baby & Polo G),677KTHp39ZBJ9csDdgymI9,0.835,0.482,8,-8.607,0,0.3670,0,0.0991,0.448,143.015,169532,4,76.0,billboard_100
147,HIXTAPE,HIXTAPE: Vol. 1,"One Beer (HARDY feat. Lauren Alaina, Devin Daw...",5FmvaZGd6fulojSToozLRD,0.516,0.745,2,-8.352,1,0.0774,0,0.2750,0.825,186.004,173173,3,77.0,billboard_100
148,Pop Smoke,Shoot For The Stars Aim For The Moon,44 BullDog,7sjFIZ1g5QLJLGja3k592K,0.800,0.564,8,-5.817,1,0.4000,1.65e-06,0.1120,0.318,139.989,150857,4,81.0,billboard_100


In [193]:
sp.playlist("5ABHKGoOzxkaa28ttQV9sE")

{'collaborative': False,
 'description': 'The 100 most streamed songs on spotify ever, in correct order! (Updated July 19th 2020)',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/5ABHKGoOzxkaa28ttQV9sE'},
 'followers': {'href': None, 'total': 4161},
 'href': 'https://api.spotify.com/v1/playlists/5ABHKGoOzxkaa28ttQV9sE?additional_types=track',
 'id': '5ABHKGoOzxkaa28ttQV9sE',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab67706c0000bebb461d30b3f1d2e77f02913a6b',
   'width': None}],
 'name': 'Top 100 Most Streamed Songs On Spotify (Updated)',
 'owner': {'display_name': 'Jonathan Holgersson',
  'external_urls': {'spotify': 'https://open.spotify.com/user/jonathan.holgersson'},
  'href': 'https://api.spotify.com/v1/users/jonathan.holgersson',
  'id': 'jonathan.holgersson',
  'type': 'user',
  'uri': 'spotify:user:jonathan.holgersson'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MjI3MCxkN2E1OGJkMjQzM2NmYjQ0ODA1YjZhYWQ3NGNhZDAyZGNmNzU4OThk',

In [149]:
#analyze_playlist("Billboard", "6UeSakyzhiEt4NB3UAd6NQ")

In [194]:
for i, (key, val) in enumerate(playlist_dict.items()):
    print (i, key, val)

0 today_top_hits ('spotify', '37i9dQZF1DXcBWIGoYBM5M')
1 billboard_100 ('Billboard', '6UeSakyzhiEt4NB3UAd6NQ')


In [195]:
playlist_dict.items()

dict_items([('today_top_hits', ('spotify', '37i9dQZF1DXcBWIGoYBM5M')), ('billboard_100', ('Billboard', '6UeSakyzhiEt4NB3UAd6NQ'))])

In [196]:
multiple_playlist_df.count()

artist              150
album               150
track_name          150
track_id            150
danceability        150
energy              150
key                 150
loudness            150
mode                150
speechiness         150
instrumentalness    150
liveness            150
valence             150
tempo               150
duration_ms         150
time_signature      150
popularity          150
playlist            150
dtype: int64

In [197]:
multiple_playlist_df.mean()

danceability             0.699627
energy                   0.635553
key                      5.446667
loudness                -6.326167
mode                     0.613333
speechiness              0.112137
instrumentalness         0.002452
liveness                 0.187655
valence                  0.501374
tempo                  118.537740
duration_ms         190580.553333
time_signature           3.973333
popularity              83.340000
dtype: float64

In [198]:
multiple_playlist_df.corr()

,danceability,energy,loudness,speechiness,liveness,valence,tempo,popularity
danceability,1.000000,-0.163942,-0.210510,0.367322,-0.080162,0.301824,0.008926,0.154378
energy,-0.163942,1.000000,0.718929,-0.112515,0.059759,0.388129,-0.014583,-0.168789
loudness,-0.210510,0.718929,1.000000,-0.043783,0.030696,0.264271,0.038869,-0.141729
speechiness,0.367322,-0.112515,-0.043783,1.000000,-0.134169,-0.065863,0.107851,-0.003346
liveness,-0.080162,0.059759,0.030696,-0.134169,1.000000,0.012525,0.125802,0.118675
valence,0.301824,0.388129,0.264271,-0.065863,0.012525,1.000000,-0.013913,0.040313
tempo,0.008926,-0.014583,0.038869,0.107851,0.125802,-0.013913,1.000000,-0.030350
popularity,0.154378,-0.168789,-0.141729,-0.003346,0.118675,0.040313,-0.030350,1.000000


In [199]:
multiple_playlist_df["track_id"].value_counts()

5f1joOtoMeyppIcJGZQvqJ    2
1Cv1YLb4q0RzL6pybtaMLo    2
4HBZA5flZLE435QTztThqH    2
6wJYhPfqk3KGhHRG76WzOh    2
7ce20yLkzuXXLUhzIDoZih    2
                         ..
5lcIegmdblnWqT9nS9EhZj    1
7sjFIZ1g5QLJLGja3k592K    1
1raaNykBg1bDnWENUiglUA    1
7iIzwqG5UxPYwTYUUK4pzU    1
7xbWAw3LMgRMn4omR5yVn3    1
Name: track_id, Length: 128, dtype: int64

In [200]:
len(multiple_playlist_df["track_id"].unique())

128

In [201]:
sorted_df = multiple_playlist_df.sort_values("track_id")
sorted_df
duplicates_df = sorted_df[sorted_df.duplicated(["track_id"], keep="first")]
duplicates_df
#[merge_df.duplicated(["Mouse ID", "Timepoint"], keep=False)]

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,playlist
28,Drake,Toosie Slide,Toosie Slide,127QTOFJsJQp5LbJbu3A1y,0.834,0.454,1,-9.750,0,0.2010,6.15e-06,0.1140,0.8370,81.618,247059,4,92.0,today_top_hits
21,Black Eyed Peas,MAMACITA,MAMACITA,14wf185UxfNbSy8dwt4r4q,0.894,0.635,2,-6.929,1,0.0473,0.00105,0.0823,0.4280,105.013,249143,4,90.0,today_top_hits
94,Surfaces,Where the Light Is,Sunday Best,1Cv1YLb4q0RzL6pybtaMLo,0.878,0.525,5,-6.832,1,0.0578,0,0.0714,0.6940,112.022,158571,4,92.0,billboard_100
101,Rod Wave,Pray 4 Love,Rags2Riches (feat. ATR Son Son),1ENdcyhqdylaLsPVgB83qq,0.904,0.252,5,-17.540,0,0.1100,0,0.1040,0.2270,117.994,167184,4,85.0,billboard_100
97,Pop Smoke,Shoot For The Stars Aim For The Moon,The Woo (feat. 50 Cent & Roddy Ricch),1H7KnK26kc1YyellpbINEn,0.490,0.618,1,-5.655,1,0.1040,3.77e-06,0.2590,0.2860,99.700,201600,4,89.0,billboard_100
11,Chris Brown,Slime & B,Go Crazy,1IIKrJVP1C9N7iPtG6eOsK,0.755,0.578,0,-8.932,0,0.1450,0,0.2500,0.5810,94.148,176960,4,88.0,today_top_hits
77,Lady Gaga,Rain On Me (with Ariana Grande),Rain On Me (with Ariana Grande),24ySl2hOPGCDcxBxFIqWBu,0.672,0.855,9,-3.764,1,0.0397,0,0.3230,0.6460,123.056,182200,4,93.0,billboard_100
52,Jack Harlow,"WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...","WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...",2MbdDtCv5LUVjYy9RuGTgC,0.904,0.723,11,-5.224,0,0.2600,0,0.1850,0.8350,145.013,227478,4,90.0,billboard_100
15,Marshmello,Be Kind (with Halsey),Be Kind (with Halsey),3Z8FwOEN59mRMxDCtb8N0A,0.630,0.633,4,-7.088,1,0.0457,0,0.0541,0.4520,93.995,172762,4,91.0,today_top_hits
12,Ariana Grande,Stuck with U,Stuck with U (with Justin Bieber),4HBZA5flZLE435QTztThqH,0.597,0.450,8,-6.658,1,0.0418,0,0.3820,0.5370,178.765,228482,3,93.0,today_top_hits


In [202]:
cleaned_top_df = multiple_playlist_df.drop(duplicates_df.index)
cleaned_top_df

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,playlist
1,DJ Khaled,POPSTAR (feat. Drake),POPSTAR (feat. Drake),6EDO9iiTtwNv6waLwa1UUq,0.800,0.560,5,-4.818,0,0.2610,0,0.1340,0.4500,163.071,200221,4,86.0,today_top_hits
2,Harry Styles,Fine Line,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,0.548,0.816,0,-4.209,1,0.0465,0,0.3350,0.5570,95.390,174000,4,95.0,today_top_hits
3,Jawsh 685,Savage Love (Laxed - Siren Beat),Savage Love (Laxed - Siren Beat),5T490vvoFNU6psep0NPmxs,0.795,0.480,3,-8.556,1,0.0688,0,0.2710,0.7230,149.919,171375,4,87.0,today_top_hits
4,Jack Harlow,"WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...","WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil W...",2MbdDtCv5LUVjYy9RuGTgC,0.904,0.723,11,-5.224,0,0.2600,0,0.1850,0.8350,145.013,227478,4,90.0,today_top_hits
5,Juice WRLD,Legends Never Die,Wishing Well,6o3QUC5oAE4g6WxRIFcZtb,0.650,0.613,9,-6.130,0,0.1280,0,0.2670,0.0804,149.972,194621,4,91.0,today_top_hits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Kenny Chesney,Knowing You,Here And Now,02LPxsehaIst1MW8wLtiyt,0.509,0.814,9,-3.576,1,0.0532,0,0.1200,0.7120,148.034,171493,4,23.0,billboard_100
146,Lil Durk,3 Headed Goat (feat. Lil Baby & Polo G),3 Headed Goat (feat. Lil Baby & Polo G),677KTHp39ZBJ9csDdgymI9,0.835,0.482,8,-8.607,0,0.3670,0,0.0991,0.4480,143.015,169532,4,76.0,billboard_100
147,HIXTAPE,HIXTAPE: Vol. 1,"One Beer (HARDY feat. Lauren Alaina, Devin Daw...",5FmvaZGd6fulojSToozLRD,0.516,0.745,2,-8.352,1,0.0774,0,0.2750,0.8250,186.004,173173,3,77.0,billboard_100
148,Pop Smoke,Shoot For The Stars Aim For The Moon,44 BullDog,7sjFIZ1g5QLJLGja3k592K,0.800,0.564,8,-5.817,1,0.4000,1.65e-06,0.1120,0.3180,139.989,150857,4,81.0,billboard_100


In [203]:
cleaned_top_df.mean()

danceability             0.695156
energy                   0.642102
key                      5.437500
loudness                -6.166203
mode                     0.617188
speechiness              0.113857
instrumentalness         0.002850
liveness                 0.186191
valence                  0.505177
tempo                  119.273672
duration_ms         189649.250000
time_signature           3.976562
popularity              82.078125
dtype: float64